For sharing with others, I adapted this program to run on kaggle, but you would probably be better off converting training images elsewhere.

The purpose of this program is to create a set of images which are composed of a set of image tiles, and to then fill in most whitespace between tiles with other portions of the image.

This program uses image tile generation methods that I found on Kaggle, (https://www.kaggle.com/rftexas/better-image-tiles-removing-white-spaces).  Then it uses my methods to remove most of the remaining white space.  It replaces the white space with other portions of the image.  The recipe I used to accomplish this is crude and inefficient.  It could be made a lot smarter about defining what white space is, knowing where it resides, and using fewer loops to overwrite it.

I have an earlier version of the program that does not use tiles.  It simply copies non-whitespace data into white space areas, in the hope that most images will eventually have little to no white space, but it doesn't check for that.

Some of the code below is unused and is not needed here, as I just borrowed sections of code from an early version of one of our competition notebooks.


In [1]:
import numpy as np
import pandas as pd
import os

import time
import gc
import cv2
import PIL
import random
import openslide
import skimage.io
from skimage.transform import rescale, resize, downscale_local_mean
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image, display

# from torch import cuda

from multiprocessing import Pool

# import cupy as cp
""" My first version worked ok with cupy and multiprocessing, but 
    In this program, I encountered known limitations that I didn't try to fix 
"""

" My first version worked ok with cupy and multiprocessing, but \n    In this program, I encountered known limitations that I didn't try to fix \n"

In [2]:
from PIL import Image

In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai import *
from fastai.vision import *

In [4]:
train = pd.read_csv('/kaggle/input/prostate-cancer-grade-assessment/train.csv')
train[['primary Gleason', 'secondary Gleason']] = train.gleason_score.str.split('+',expand=True)
display(train.head())

test = pd.read_csv('/kaggle/input/prostate-cancer-grade-assessment/test.csv')
display(test)

submission = pd.read_csv('/kaggle/input/prostate-cancer-grade-assessment/sample_submission.csv')
submission

train['id'] = train['image_id'] + '.jpeg'
train_isup = train[['id', 'isup_grade']]
train_primary = train[['id', 'primary Gleason']]
train_secondary = train[['id', 'secondary Gleason']]

,image_id,data_provider,isup_grade,gleason_score,primary Gleason,secondary Gleason
0,0005f7aaab2800f6170c399693a96917,karolinska,0,0+0,0,0
1,000920ad0b612851f8e01bcc880d9b3d,karolinska,0,0+0,0,0
2,0018ae58b01bdadc8e347995b69f99aa,radboud,4,4+4,4,4
3,001c62abd11fa4b57bf7a6c603a11bb9,karolinska,4,4+4,4,4
4,001d865e65ef5d2579c190a0e0350d8f,karolinska,0,0+0,0,0


,image_id,data_provider
0,005700be7e06878e6605e7a5a39de1b2,radboud
1,005c6e8877caf724c600fdce5d417d40,karolinska
2,0104f76634ff89bfff1ef0804a95c380,radboud


In [5]:
images = list(train['image_id'])
images.append(list(test['image_id']))
labels = list(train['isup_grade'])

In [6]:
if not os.path.exists('/kaggle/working/images-tiled-striplicated'):
    os.mkdir('images-tiled-striplicated')

In [7]:
test_df = pd.read_csv('../input/prostate-cancer-grade-assessment/test.csv')
sample_submission = pd.read_csv('../input/prostate-cancer-grade-assessment/sample_submission.csv')

In [8]:
data_dir = '../input/prostate-cancer-grade-assessment/train_images/'

# compute_statistics was copied from: 
#    https://www.kaggle.com/rftexas/better-image-tiles-removing-white-spaces
def compute_statistics(image):
    """
    Args:
        image                  numpy.array   multi-dimensional array of the form WxHxC
    
    Returns:
        ratio_white_pixels     float         ratio of white pixels over total pixels in the image 
    """
    width, height = image.shape[0], image.shape[1]
    num_pixels = width * height
    
    num_white_pixels = 0
    
    summed_matrix = np.sum(image, axis=-1)
    # Note: A 3-channel white pixel has RGB (255, 255, 255)
    num_white_pixels = np.count_nonzero(summed_matrix > 620)
    ratio_white_pixels = num_white_pixels / num_pixels
    
    green_concentration = np.mean(image[1])
    blue_concentration = np.mean(image[2])
    
    return ratio_white_pixels, green_concentration, blue_concentration


# select_k_best_regions was copied from: 
#    https://www.kaggle.com/rftexas/better-image-tiles-removing-white-spaces
def select_k_best_regions(regions, k=20):
    """
    Args:
        regions               list           list of 2-component tuples first component the region, 
                                             second component the ratio of white pixels
                                             
        k                     int            number of regions to select
    """
    regions = [x for x in regions if x[3] > 180 and x[4] > 180]
    k_best_regions = sorted(regions, key=lambda tup: tup[2])[:k]
    return k_best_regions

# get_k_best_regions was copied from: 
#    https://www.kaggle.com/rftexas/better-image-tiles-removing-white-spaces
def get_k_best_regions(coordinates, image, window_size=1024):#window size 512 is default
    regions = {}
    for i, tup in enumerate(coordinates):
        x, y = tup[0], tup[1]
        regions[i] = image[x : x+window_size, y : y+window_size, :]
    
    return regions

# generate_patches was copied from: 
#    https://www.kaggle.com/rftexas/better-image-tiles-removing-white-spaces
def generate_patches(slide_path, window_size=200, stride=256, k=20):#stride 128
    
    image = skimage.io.MultiImage(slide_path)[-2]
    image = np.array(image)
    
    max_width, max_height = image.shape[0], image.shape[1]
    regions_container = []
    i = 0
    
    while window_size + stride*i <= max_height:
        j = 0
        
        while window_size + stride*j <= max_width:            
            x_top_left_pixel = j * stride
            y_top_left_pixel = i * stride
            
            patch = image[
                x_top_left_pixel : x_top_left_pixel + window_size,
                y_top_left_pixel : y_top_left_pixel + window_size,
                :
            ]
            
            ratio_white_pixels, green_concentration, blue_concentration = compute_statistics(patch)
            
            region_tuple = (x_top_left_pixel, y_top_left_pixel, ratio_white_pixels, green_concentration, blue_concentration)
            regions_container.append(region_tuple)
            
            j += 1
        
        i += 1
    
    k_best_region_coordinates = select_k_best_regions(regions_container, k=k)
    k_best_regions = get_k_best_regions(k_best_region_coordinates, image, window_size)
    
    return image, k_best_region_coordinates, k_best_regions


# display_images was copied from: 
#    https://www.kaggle.com/rftexas/better-image-tiles-removing-white-spaces
def display_images(regions, title):
    fig, ax = plt.subplots(5, 4, figsize=(15, 15))
    
    for i, region in regions.items():
        ax[i//4, i%4].imshow(region)
    
    fig.suptitle(title)
    

# glue_to_one_picture was copied from: 
#    https://www.kaggle.com/rftexas/better-image-tiles-removing-white-spaces
def glue_to_one_picture(image_patches, window_size=200, k=32):
    side = int(np.sqrt(k))
    image = np.zeros((side*window_size, side*window_size, 3), dtype=np.int16)
        
    for i, patch in image_patches.items():
        x = i // side
        y = i % side
        image[
            x * window_size : (x+1) * window_size,
            y * window_size : (y+1) * window_size,
            :
        ] = patch
    
    return image

In [9]:
def crop_center(img,cropx,cropy):
    y,x,z = img.shape
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)    
    return img[starty:starty+cropy,startx:startx+cropx,:z]

In [10]:
def slur(image, n):
    """
    make a copy of the image, shift and write wherever there is white space.
    """

    r1, g1, b1 = 214, 214, 214 
    r2, g2, b2 = 255, 255, 255
    red, green, blue = image[n:,:,0], image[n:,:,1], image[n:,:,2]
    mask = (red >= r1) & (green >= g1) & (blue >= b1)
    
    # not sure why I wrote the next line of code.  Perhaps I was low on caffeine
    # I'll think later, but this looks buggly.
    image[n:,:,:3][mask] = [r2, g2, b2]
    
    slurred = np.copy(image)
#     slurred = cp.copy(image)
#     cp.cuda.Stream.null.synchronize()

    
    if n >= 0:
        slurred[n:,:,:3][mask] = image[:-n,:,:3][mask]
        
    # at one point there was a horrible memory leak, so I started
    # calling del. Something made the problem go away, but I didn't root cause it.
    del image
    del mask
    del red
    del green
    del blue
    print('.', end='')
    return slurred

In [11]:
def striplicate(npimage):
    """
    Set whiteness in r1,g1,g1
    
    logically divide the image into stripes.
    stripes are grouped columns of stripe_size, 
    which is cols divided by some smaller number.
        
    make a rotated copy of an image, and then use that to
    kickstart the process of slur()'ing over the white space.
    
    """
    
    # image = cp.copy(npimage)
    image = np.copy(npimage)
    # cp.cuda.Stream.null.synchronize()
    # help(cp.cuda.Stream.null)
    rimage = np.rot90(image, k=1)
    # rimage = cp.rot90(image, k=1)
    # cp.cuda.Stream.null.synchronize()
        
    r1, g1, b1 = 212, 212, 212 # Original value for whiteness
    r2, g2, b2 = 255, 255, 255 # Value to replace it with
   
    red, green, blue = image[:,:,0], image[:,:,1], image[:,:,2]
    rred, rgreen, rblue = rimage[:,:,0], rimage[:,:,1], rimage[:,:,2]
    
    mask = (red >= r1) & (green >= g1) & (blue >= b1)
    rmask =(rred >= r1) & (rgreen >= g1) & (rblue >= b1)
        
    rimage[:,:,:3][rmask] = [r2, g2, b2]
    del rmask
    rows,cols,depth = rimage.shape

    stripe_size = cols//6
    simage = None
    for stripe in range(stripe_size, cols, stripe_size):
        # print(f'stripe: {stripe}, cols: {cols}')
        if simage is not None:
            simage = slur(simage, stripe_size)
        else:
            simage = slur(rimage, stripe_size)
            del rimage
            
    stripe_size = cols//3
    
    simage = np.rot90(simage, k=1)
    # simage = cp.rot90(simage, k=1)
    # cp.cuda.Stream.null.synchronize()

    for stripe in range(stripe_size, cols, stripe_size):
        # print(f'stripe: {stripe}, cols: {cols}')
        if simage is not None:
            simage = slur(simage, stripe_size)
        else:
            simage = slur(rimage, stripe_size)
            del rimage
            
    stripe_size = cols//2      
    simage = np.rot90(simage, k=1)
    # simage = cp.rot90(simage, k=1)
    # cp.cuda.Stream.null.synchronize()

    for stripe in range(stripe_size, cols, stripe_size):
        # print(f'stripe: {stripe}, cols: {cols}')
        if simage is not None:
            simage = slur(simage, stripe_size)
        else:
            simage = slur(rimage, stripe_size)
            del rimage
    
    stripe_size = cols//2
    simage = np.rot90(simage, k=1)
    # simage = cp.rot90(simage, k=1)
    # cp.cuda.Stream.null.synchronize()
    for stripe in range(stripe_size, cols, stripe_size):
        # print(f'stripe: {stripe}, cols: {cols}')
        if simage is not None:
            simage = slur(simage, stripe_size)
        else:
            simage = slur(rimage, stripe_size)
            del rimage

    stripe_size = cols//2
    simage = np.rot90(simage, k=1)
    # simage = cp.rot90(simage, k=1)
    # cp.cuda.Stream.null.synchronize()

    for stripe in range(stripe_size, cols, stripe_size):
        # print(f'stripe: {stripe}, cols: {cols}')
        if simage is not None:
            simage = slur(simage, stripe_size)
        else:
            simage = slur(rimage, stripe_size)
            del rimage

            
    print(f'simage shape: {simage.shape}')


    image[:,:,:3][mask] = [r2, g2, b2]
    image[:,:,:3][mask] = simage[:,:,:3][mask]

    return image
#     return crop_center(image, 2048, 2048)

In [12]:
WINDOW_SIZE = 256
STRIDE = 128
K = 4

In [13]:
def process_images(range_list):
    print(f'range_list: {range_list}')
    starting_index, ending_index = range_list

    for counter, img in enumerate(images):
        if counter > 16:  # limit this for test purposes.
            continue

        if counter < starting_index or counter > ending_index or os.path.isfile(f'images-tiled-striplicated/{img}.jpeg'):
            continue
#         print(f'{counter}', end=', ')

        
     
        fig, ax = plt.subplots(6, 2, figsize=(20, 25))

        url = data_dir + img + '.tiff'
        print(url)
        
        image, best_coordinates, best_regions = generate_patches(url, window_size=WINDOW_SIZE, stride=STRIDE, k=K)
        
        glued_image = glue_to_one_picture(best_regions, window_size=WINDOW_SIZE, k=K)
        simage = striplicate(np.array(glued_image, dtype=np.uint8))
        npImage = np.array(simage, dtype=np.uint8)

        plt.imsave(f'images-tiled-striplicated/{img}.jpeg', npImage, format='jpg')
        
        plt.close()
        del npImage
        del glued_image
        del fig
        del ax
        del best_coordinates
        del best_regions
        gc.collect()

In [14]:
# This will take a very long time.
nprocs=6   # nprocs over 16, swamps my home computer.
end=10620  # just some number a little bigger than total number of images.
group_len = end//nprocs
start=0
# multiprocessing.set_start_method('spawn')
# ctx = multiprocessing.get_context('spawn')
# multiprocessing.set_start_method('fork')

range_list = list()

for i in range(1,nprocs+1):
    end = group_len * i
    range_list.append([start, end])
    start = end

with Pool(nprocs) as workers:
    w = list(workers.map(process_images, range_list))
    print(w)
    
print("it's Miller time!")

range_list: [5310, 7080]
range_list: [3540, 5310]
range_list: [0, 1770]
range_list: [1770, 3540]
range_list: [7080, 8850]
range_list: [8850, 10620]
../input/prostate-cancer-grade-assessment/train_images/0005f7aaab2800f6170c399693a96917.tiff
............simage shape: (512, 512, 3)
../input/prostate-cancer-grade-assessment/train_images/000920ad0b612851f8e01bcc880d9b3d.tiff
............simage shape: (512, 512, 3)
../input/prostate-cancer-grade-assessment/train_images/0018ae58b01bdadc8e347995b69f99aa.tiff
............simage shape: (512, 512, 3)
../input/prostate-cancer-grade-assessment/train_images/001c62abd11fa4b57bf7a6c603a11bb9.tiff
............simage shape: (512, 512, 3)
../input/prostate-cancer-grade-assessment/train_images/001d865e65ef5d2579c190a0e0350d8f.tiff
............simage shape: (512, 512, 3)
../input/prostate-cancer-grade-assessment/train_images/002a4db09dad406c85505a00fb6f6144.tiff
............simage shape: (512, 512, 3)
../input/prostate-cancer-grade-assessment/train_images